## Group Assignment

#### Import the necessary libraries

In [194]:
import numpy as np
import pandas as pd

import datetime as dt
import yfinance as yf
import pandas_datareader.data as web

import matplotlib.pyplot as plt
%matplotlib inline

#### All questions in Parts I and II apply to a random sample of 15 stocks that your group will be assigned by running the following code.

#### Random Sample Selection 

In [195]:
np.random.seed (2051 + 6)
ticker_list = ['AAPL', 'AXP', 'BA', 'C','CAT', 'CSCO', 'CVX', 'DIS', 'GS', 'HD', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO',
            'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PFE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WMT', 'XOM']

stock_list = np.random.choice(ticker_list,15,replace=False)
print(f'These are the fifteen stocks assigned to you: {" ".join(stock_list)}')

These are the fifteen stocks assigned to you: DIS PG V UNH CSCO PFE JPM AXP JNJ INTC KO CAT WMT CVX XOM


### PART I

#### 1. Find the optimal portfolio over the period January 2015 - December 2019, using the fifteen stocks assigned to your group. Assume there are no short-selling constraints.

In [196]:
start = dt.datetime(2015,1,1)
end = dt.datetime(2019,12,31)

stock_list=['DIS', 'PG', 'V', 'UNH', 'CSCO', 'PFE', 'JPM', 'AXP', 'JNJ', 'INTC', 'KO', 'CAT', 'WMT', 'CVX', 'XOM']

returns = yf.download(stock_list,start-pd.offsets.BDay(1),end)['Adj Close'].pct_change().dropna()

[*********************100%***********************]  15 of 15 completed


In [197]:
from scipy.optimize import minimize

In [198]:
returns_mon = returns.resample('M').apply(lambda x: x.add(1).prod().sub(1))

In [199]:
rf = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0][['RF']].div(100)
rf.index = rf.index.to_timestamp(how='end').normalize()

In [200]:
def port_ret(weights):
    port_ret = np.dot(returns_mon*12,weights).mean()
    return port_ret

def port_std(weights):
    port_std = np.sqrt(np.dot(weights, np.dot(returns_mon.cov()*12, weights)))
    return port_std

def ex_port_ret(weights):
    ex_port_ret = (np.dot(returns_mon,weights) - rf['RF']).mean()*12
    return ex_port_ret

def ex_port_std(weights):
    ex_port_std = (np.dot(returns_mon,weights) - rf['RF']).std()*np.sqrt(12)
    return ex_port_std

def neg_SR(weights):
    SR = ex_port_ret(weights) / ex_port_std(weights)
    return (-1)*SR

In [201]:
constraints = ({'type':'eq','fun': lambda weights: np.sum(weights) - 1})

init_guess = np.full(len(returns_mon.columns), 1/len(returns_mon.columns))

In [202]:
optimal_port=minimize(neg_SR,init_guess,constraints=constraints)
optimal_port

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -2.319871475703662
       x: [-4.875e-01  2.980e-04 ...  4.360e-02 -5.634e-01]
     nit: 16
     jac: [-5.631e-02 -5.625e-02 ... -5.587e-02 -5.629e-02]
    nfev: 259
    njev: 16

#### 2. What are the weights of the stocks in the optimal portfolio?

In [203]:
for ticker in stock_list:
    result=print(ticker + "  "+ str(round(optimal_port.x[stock_list.index(ticker)],4)))

DIS  -0.4875
PG  0.0003
V  0.0209
UNH  0.2153
CSCO  -0.1971
PFE  0.0582
JPM  0.0245
AXP  0.5187
JNJ  -0.1174
INTC  -0.2953
KO  0.3236
CAT  0.594
WMT  0.8617
CVX  0.0436
XOM  -0.5634


#### 3. What was the annualized average monthly return for the optimal portfolio?

In [204]:
optimal_weights = optimal_port.x
optimal_annual_return = port_ret(optimal_weights) * 12

#### 4. What was the annualized monthly standard deviation for the optimal portfolio?

In [205]:
optimal_std=port_std(optimal_weights) * 12

#### 5. What was the Sharpe Ratio of the optimal portfolio?

In [206]:
-optimal_port.fun

2.319871475703662

### PART II

#### 1. Create a DataFrame named optimal_weights to store the weights from a portfolio optimization performed on a rolling basis. Specifically, the optimization should use a 60 month rolling window, and be performed every month. The first 60 months correspond to the sample period for data_initial (01/2015 - 12/2019) created in question 3. Hence, the first observation in the optimal_weights DataFrame should be the weights you listed above in Part I. The next 60 month period should be 02/2015 - 01/2020, followed by 03/2015 - 02/2020, and so on. The last 60 month period should be 03/2018 - 02/2023. Again, assume there are no short-selling constraints.

In [207]:
start = dt.datetime(2015,1,1)
end = dt.datetime(2023,2,28)

stock_list=['DIS', 'PG', 'V', 'UNH', 'CSCO', 'PFE', 'JPM', 'AXP', 'JNJ', 'INTC', 'KO', 'CAT', 'WMT', 'CVX', 'XOM']

returns = yf.download(stock_list,start-pd.offsets.BDay(1),end + pd.offsets.BDay(1))['Adj Close'].pct_change().dropna()

[*********************100%***********************]  15 of 15 completed


In [208]:
rf_rate = web.DataReader('F-F_Research_Data_Factors','famafrench', start, end)[0][['RF']].div(100)

In [209]:
returns_mon = returns.resample('M').apply(lambda x: x.add(1).prod().sub(1))

In [210]:
optimal_weights = pd.DataFrame()

for i in range(0,len(returns_mon.index)-60+1):
    # Define the negative Sharpe Ratio function that we will minimize

    def neg_SR(weights):
        SR = (np.dot(returns_mon.iloc[i:i+60],weights) - rf_rate.iloc[i:i+60]['RF']).mean()*12 /\
        (np.dot(returns_mon.iloc[i:i+60],weights) - rf_rate.iloc[i:i+60]['RF']).std()*np.sqrt(12)
        return (-1)*SR
    
    constraints = ({'type':'eq','fun': lambda weights: np.sum(weights) - 1})

    init_guess = np.full(len(returns_mon.columns),1/len(returns_mon.columns))
    
    optimal_port = minimize(neg_SR,init_guess,constraints = constraints)
    
    optimal_weights = pd.concat([optimal_weights,
                                pd.DataFrame(optimal_port.x.reshape(1,len(stock_list)).round(4),
                                             columns=[stock_list],
                                             index=[returns_mon.iloc[i:i+60].index[-1]])],
                                axis=0)
    
optimal_weights

,DIS,PG,V,UNH,CSCO,PFE,JPM,AXP,JNJ,INTC,KO,CAT,WMT,CVX,XOM
2019-12-31,-0.4877,-0.0011,0.0226,0.2158,-0.1959,0.0590,0.0262,0.5187,-0.1166,-0.2934,0.3239,0.5925,0.8572,0.0411,-0.5624
2020-01-31,-0.2830,0.0031,-0.0477,0.1757,-0.2099,0.0982,0.0723,0.4988,-0.0484,-0.2698,0.3630,0.4233,0.7683,0.0120,-0.5559
2020-02-29,-0.4572,0.0761,-0.0546,0.3099,-0.3498,0.0655,0.2111,0.6750,-0.1771,-0.4920,0.4167,0.5720,1.0763,0.0274,-0.8993
2020-03-31,-0.4561,0.1217,0.0187,0.2609,-0.3704,0.0801,0.2333,0.6310,-0.1347,-0.4965,0.4318,0.5301,1.0245,0.0562,-0.9307
2020-04-30,-0.3312,0.0221,-0.0799,0.2386,-0.3167,0.0915,0.2328,0.4803,-0.2548,-0.3669,0.4609,0.5522,0.9469,0.0813,-0.7571
2020-05-31,-0.3419,0.0299,-0.0379,0.3077,-0.2989,0.0725,0.2342,0.4599,-0.2528,-0.4032,0.4294,0.5327,0.9609,0.1150,-0.8075
2020-06-30,-0.2989,0.0411,-0.0000,0.3963,-0.3274,0.0962,0.2331,0.3982,-0.2227,-0.4535,0.4122,0.4862,0.9364,0.1523,-0.8497
2020-07-31,-0.2335,0.1016,-0.0100,0.3111,-0.3175,0.0292,0.0993,0.3531,-0.1395,-0.2617,0.4896,0.4103,0.7641,0.1694,-0.7656
2020-08-31,-0.3430,0.0659,0.0977,0.3576,-0.1855,0.0214,0.0388,0.3413,-0.0822,-0.1858,0.4342,0.3481,0.6394,0.1984,-0.7466
2020-09-30,-0.3035,0.1678,0.0383,0.3441,-0.2450,-0.0245,0.0462,0.3681,-0.1350,-0.1727,0.4590,0.3231,0.6913,0.2201,-0.7773


In [187]:
returns_mon = returns.resample('M').apply(lambda x: x.add(1).prod().sub(1))

In [188]:
returns_mon.shape

(98, 15)

In [192]:
optimal_weights = pd.DataFrame()

for i in range(0,len(returns_mon.index)-60+1):
    window = returns_mon.iloc[i:i+60]

    rf = web.DataReader('F-F_Research_Data_Factors','famafrench', i, i+60)[0][['RF']].div(100)
    rf.index = rf.index.to_timestamp(how='end').normalize()

    def port_ret(weights):
        port_ret = np.dot(window*12,weights).mean()
        return port_ret
    
    def port_std(weights):
        port_std = np.sqrt(np.dot(weights, np.dot(window.cov()*12, weights)))
        return port_std
    
    def ex_port_ret(weights):
        ex_port_ret = (np.dot(window,weights) - rf['RF']).mean()*12
        return ex_port_ret
    
    def ex_port_std(weights):
        ex_port_std = (np.dot(window,weights) - rf['RF']).std()*np.sqrt(12)
        return ex_port_std
    
    def neg_SR(weights):
        SR = ex_port_ret(weights) / ex_port_std(weights)
        return (-1)*SR
    

    constraints = ({'type':'eq','fun': lambda weights: np.sum(weights) - 1})

    init_guess = np.full(len(returns_mon.columns),1/len(returns_mon.columns))
    
    optimal_port = minimize(neg_SR,init_guess,constraints = constraints)
    
    optimal_weights = pd.concat([optimal_weights,
                                pd.DataFrame(optimal_port.x.reshape(1,len(stock_list)).round(4),
                                             columns=[stock_list],
                                             index=[returns_mon.iloc[i:i+60].index[-1]])],
                                axis=0)
    
optimal_weights


OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 60-01-01 00:00:00 present at position 0

#### 2. Create a new DataFrame named port_returns to store the monthly returns over the sample period 01/2020 - 02/2023 for the following portfolios:
 1) A monthly rebalanced portfolio using the rolling optimal weights. Specifically, the portfolio return for 01/2020 should be based on the weights as of 12/2019 (the same values detailed in Part I and in the first row in the optimal_weights DataFrame), the portfolio return for 02/2020 should be based on the weights as of 01/2020 and so on. Label the portfolio (MRoll_Reb_OP, an acronym for Monthly Rolling Rebalanced Optimal Portfolio). 
 2) A monthly rebalanced portfolio using the optimal weights as of 12/2019. Label the portfolio (M_Reb_OP, an acronym for Monthly Rebalancing Optimal Portfolio).
 3) A monthly rebalanced equally-weighted portfolio. Label the portfolio (M_Reb_EW, an acronym for Monthly Rebalancing Equally-Weighted Portfolio).
 4) A buy and hold portfolio, initially allocated according to the optimal weights as of 12/2019. Label the portfolio (BH_OP, an acronym for Buy and Hold Optimal Portfolio).
 5) A buy and hold portfolio, initially allocated equally across stocks. Label the portfolio (BH_EW, an acronym for Buy and Hold Equally-Weighted Portfolio).

#### 3. Create a DataFrame port_stats (with a 3x5 shape) to store the annualized average monthly return, the annualized monthly standard deviation and the Sharpe ratio for all 5 portfolios.

#### 4. Compute the cumulative return series for all portfolios and plot them in the same graph. 

#### 5. What was the cumulative return for the best performing portfolio? What was the cumulative return for the worst performing portfolio?

### PART III - Examination of the Size Effect

Please download the file "crsp_fall22.csv" located in folder Group_Assignment. The file contains the following columns:
* PERMNO - Permanent number (unique identifer for the stock)
* DATE
* PRC - The closing price
* VOL - Trading volume (in hundreds)
* RET - The simple return
* SHROUT - The number of shares outstanding (in thousands)
* CFACPR - Cumulative factor to adjust the closing price
* CFACSHR - Cumulative factor to adjust shares outstanding

#### 1. Create a DataFrame named fin_data by reading in the columns DATE, PERMNO, RET, PRC and SHROUT from the file 'crsp_fall22.csv'. Set the DATE as the index. In addition, change each index value to the end of the month. For example, change '1926-05-30' to 1926-05-31', or '1987-01-30' to '1987-01-31'. In addition, create a new column (labeled MKTCAP) to store the market capitalization (defined as PRC * SHROUT). Subsequently, permanently remove the PRC and SHROUT columns.

In [16]:
fin_data=pd.read_csv('crsp_fall22.csv',
                     usecols=['DATE','PERMNO','RET','PRC','SHROUT'],
                     index_col='DATE',
                     parse_dates=True)
fin_data.index=fin_data.index+pd.offsets.MonthEnd(0)
fin_data['MKTCAP']=fin_data['PRC']*fin_data['SHROUT']
fin_data.drop(['PRC','SHROUT'],axis=1,inplace=True)
fin_data

,PERMNO,RET,MKTCAP
DATE,,,
1990-01-31,21573.0,-0.106195,5.499450e+06
1990-01-31,55160.0,-0.190476,2.067094e+03
1990-01-31,45129.0,-0.086420,1.892550e+05
1990-01-31,22250.0,-0.174684,3.117920e+05
1990-01-31,60468.0,-0.071429,2.227225e+04
...,...,...,...
2022-03-31,16400.0,-0.070234,2.676084e+05
2022-03-31,16401.0,0.424749,9.943693e+04
2022-03-31,90664.0,0.236017,4.965897e+07


#### 2. How many stocks are there in the sample? How many of these stocks are in the sample over the entire sample period?

In [17]:
fin_data.nunique()

PERMNO      18153
RET        828585
MKTCAP    1611452
dtype: int64

#### 3. Every June, split the stocks into quintiles (five groups) based on their market capitalization. The header of a DataFrame named size_qt with the resulting split can be found below. 

In [101]:
size_qt.sort_index().head()

,PERMNO,MKTCAP_QT
DATE,,
1990-06-30,10294,3
1990-06-30,65496,3
1990-06-30,10905,2
1990-06-30,32037,1
1990-06-30,66288,5


#### 4. Create a new DataFrame named data by using the merge_asof() function to merge the DataFrames fin_data and size_qt. Specifically, merge each stock's June quintile allocation with the stock's monthly returns for the next 12 months starting the following July. Subsequently, set DATE as the index in the DataFrame.

In [ ]:
data=pd.DataFrame()

#### 5. Create a new DataFrame called quintiles to store the average monthly return for each quintile. The header of the DataFrame can be found below:

In [105]:
quintiles.head()

,Q1_RET,Q2_RET,Q3_RET,Q4_RET,Q5_RET
DATE,,,,,
1990-07-31,-0.001078,-0.041629,-0.044858,-0.040050,-0.028895
1990-08-31,-0.088672,-0.111944,-0.134610,-0.132793,-0.110817
1990-09-30,-0.045567,-0.090744,-0.092048,-0.099917,-0.080715
1990-10-31,-0.071408,-0.064675,-0.069424,-0.060729,-0.039041
1990-11-30,-0.011988,0.002373,0.033031,0.070116,0.099023


#### 6. Create a bar plot of the average monthly return for all five market capitalization groups, across the entire sample.

#### 7. Create a line plot of the cumulative return series for all five market capitalization groups.